In [0]:
import requests

# Let's download some data from a Google Spreadsheet

# Publicly viewable spreadsheet. 
# https://docs.google.com/spreadsheets/d/1zDFWwWLaPu5Wmlz6VQJqGaTsl7TuleoxZYdn8B3zFS0/edit?usp=sharing

# Extract the sheetKey and the sheetName you want to access data for. 

personalDataUrl = "https://docs.google.com/spreadsheets/d/14dZQ5XLFG5-1NNGYTgclrTXlkEUiSWWiOCdkjNZaAfQ/gviz/tq?tqx=out:csv&sheet=personal_data"
moduleEnrolmentUrl = "https://docs.google.com/spreadsheets/d/14dZQ5XLFG5-1NNGYTgclrTXlkEUiSWWiOCdkjNZaAfQ/gviz/tq?tqx=out:csv&sheet=module_enrolment"
programEnrolmentUrl = "https://docs.google.com/spreadsheets/d/14dZQ5XLFG5-1NNGYTgclrTXlkEUiSWWiOCdkjNZaAfQ/gviz/tq?tqx=out:csv&sheet=program_enrolment"
preEntryUrl = "https://docs.google.com/spreadsheets/d/14dZQ5XLFG5-1NNGYTgclrTXlkEUiSWWiOCdkjNZaAfQ/gviz/tq?tqx=out:csv&sheet=pre_entry_subject"
basisAdmissionUrl = "https://docs.google.com/spreadsheets/d/14dZQ5XLFG5-1NNGYTgclrTXlkEUiSWWiOCdkjNZaAfQ/gviz/tq?tqx=out:csv&sheet=basis_of_admission"

In [0]:
# Let'd download the data as text like we downloaded the raw html data. 
import urllib
response1 = urllib.request.urlopen(personalDataUrl)
data1 = response1.read()     

response2 = urllib.request.urlopen(moduleEnrolmentUrl)
data2 = response2.read()   

response3 = urllib.request.urlopen(programEnrolmentUrl)
data3 = response3.read()

response4 = urllib.request.urlopen(preEntryUrl)
data4 = response4.read()   

response5 = urllib.request.urlopen(basisAdmissionUrl)
data5 = response5.read()   

personal_data = data1.decode('utf-8') 
module_enrolment = data2.decode('utf-8')
program_enrolment = data3.decode('utf-8')
pre_entry = data4.decode('utf-8')
basis_admission = data5.decode('utf-8')

In [0]:
# Let's convert this data into a chartable format. Something like: 
# newData = [["Sun", 5], ["Mon", 46], ["Tue", 28]]

import pandas as pd
import io
s1 = requests.get(personalDataUrl).content
s2 = requests.get(moduleEnrolmentUrl).content
s3 = requests.get(programEnrolmentUrl).content
s4 = requests.get(preEntryUrl).content
s5 = requests.get(basisAdmissionUrl).content

personalData = pd.read_csv(io.StringIO(s1.decode('utf-8')))
moduleEnrolment = pd.read_csv(io.StringIO(s2.decode('utf-8')))
programEnrolment = pd.read_csv(io.StringIO(s3.decode('utf-8')))
preEntry = pd.read_csv(io.StringIO(s4.decode('utf-8')))
basisAdmission = pd.read_csv(io.StringIO(s5.decode('utf-8')))

In [0]:
def background(row):
  if (row['basis_of_admission_descr'] == "GCE 'A' Level (English)") or (row['basis_of_admission_descr'] == "Discretionary") or (row['basis_of_admission_descr'] == "Bright A Level by A-Level Exam"):
    return "JC"
  elif (row['basis_of_admission_descr'] == "Direct Entry From Poly") or (row['basis_of_admission_descr'] == "NON-HSC") or (row['basis_of_admission_descr'] == 'Poly Diploma with Merit') or (row['basis_of_admission_descr'] == 'NON-HSC'):
    return "Poly"
  else:
    return "Others"
  

In [5]:
basisAdmission['Pre-U Background'] = 'default'

for index, row in basisAdmission.iterrows():  
  basisAdmission.loc[index, 'Pre-U Background'] = background(row)

basisAdmission.head()

,token,academic_career,career_nbr,basis_of_admission_code,basis_of_admission_descr,Pre-U Background
0,32000_1,UGRD,0,ET_20,GCE 'A' Level (English),JC
1,19700_2,UGRD,0,ET_20,GCE 'A' Level (English),JC
2,12051_3,UDEN,0,ET_20,GCE 'A' Level (English),JC
3,25996_4,UGRD,0,ET_30,Direct Entry From Poly,Poly
4,55742_5,UGRD,0,ET_30,Direct Entry From Poly,Poly


In [6]:
# Merge

result = moduleEnrolment[['token','module_code']].merge(personalData[['token', 'gender', 'year_of_birth']], left_on = 'token', right_on = 'token')
result = result.merge(programEnrolment[['token', 'academic_program_descr','program_status_descr', 'degree_descr']], left_on = 'token', right_on = 'token')
result = result.merge(preEntry[['token', 'pre_entry_course_level_descr', 'pre_entry_subject_taken_descr']],left_on = 'token', right_on = 'token')
result = result.merge(basisAdmission[['token', 'Pre-U Background']],left_on = 'token', right_on = 'token')
result.head()

,token,module_code,gender,year_of_birth,academic_program_descr,program_status_descr,degree_descr,pre_entry_course_level_descr,pre_entry_subject_taken_descr,Pre-U Background
0,32000_1,ACC2002,M,1994,Exchange (Coursework),COMPLETED PROGRAMME,Bachelor of Engineering (Materials Science and...,O LEVEL,ENGLISH LITERATURE,JC
1,32000_1,PF2105,M,1994,Exchange (Coursework),COMPLETED PROGRAMME,Bachelor of Engineering (Materials Science and...,O LEVEL,ENGLISH LITERATURE,JC
2,32000_1,SSA2209,M,1994,Exchange (Coursework),COMPLETED PROGRAMME,Bachelor of Engineering (Materials Science and...,O LEVEL,ENGLISH LITERATURE,JC
3,32000_1,LSM2251,M,1994,Exchange (Coursework),COMPLETED PROGRAMME,Bachelor of Engineering (Materials Science and...,O LEVEL,ENGLISH LITERATURE,JC
4,32000_1,LAJ2203,M,1994,Exchange (Coursework),COMPLETED PROGRAMME,Bachelor of Engineering (Materials Science and...,O LEVEL,ENGLISH LITERATURE,JC


In [0]:
result.to_csv('studentInfo.csv')

from google.colab import files
files.download("studentInfo.csv")

In [0]:
# And use your own Firebase project
# Replace this with your firebase project url

studentFirebase = "https://myfirebase-e83f0.firebaseio.com/"
moduleFirebase = "https://modulefirebase-57616.firebaseio.com/"

In [0]:
import random
student = {}
module  = {}

module_list = ['BA', 'IS', 'CS', 'Info Sec', 'Statistics', 'Business', 'Econs', 'DSA', 'Maths', 'FASS']
module_num = 0

for index, row in result.iterrows():
  
  # update moduleInfo table
  if row['module_code'] not in module.keys():
    module[row['module_code']] = {}
    module[row['module_code']]['gender'] = {}
    module[row['module_code']]['gender']['F'] = random.randint(2,100)
    module[row['module_code']]['gender']['M'] = random.randint(2,100)
    
    module[row['module_code']]['major'] = {}
    module_num = random.randint(2,8)
    order_list = []
      
    while module_num > 0:
      module_order = random.randint(0,9)
      if module_order not in order_list:
        order_list.append(module_order)
        module_num -= 1
        
    for j in order_list:
      module[row['module_code']]['major'][module_list[j]] = random.randint(2,40)
      
    
    module[row['module_code']]['preU'] = {}
    module[row['module_code']]['preU']['JC'] = random.randint(10,50)
    module[row['module_code']]['preU']['poly'] = random.randint(5,30)
    module[row['module_code']]['preU']['Others'] = random.randint(2,20)
    
    
  # update studentInfo table      
  if row['token'] in student.keys():
    # update modules only
    student[row['token']]['modules'].append(row['module_code'])
      
  else:
    student[row['token']] = {}
    student[row['token']]['preU'] = row['Pre-U Background']
    student[row['token']]['major'] = row['pre_entry_subject_taken_descr']
    student[row['token']]['gender'] = row['gender']
    student[row['token']]['modules'] = []
    student[row['token']]['modules'].append(row['module_code'])
    
    
    
mocked_module_list = ['BT2101', 'BT2102', 'BT3103', 'IS2101']
for k in mocked_module_list:
  module[k] = {}
  module[k]['gender'] = {}
  module[k]['gender']['F'] = random.randint(2,100)
  module[k]['gender']['M'] = random.randint(2,100)
    
  module[k]['major'] = {}
  module_num = random.randint(2,8)
  order_list = []
      
  while module_num > 0:
    module_order = random.randint(0,9)
    if module_order not in order_list:
      order_list.append(module_order)
      module_num -= 1
        
  for j in order_list:
    module[k]['major'][module_list[j]] = random.randint(2,40)
      
    
  module[k]['preU'] = {}
  module[k]['preU']['JC'] = random.randint(10,50)
  module[k]['preU']['poly'] = random.randint(5,30)
  module[k]['preU']['Others'] = random.randint(2,20)

    

In [15]:
import json

studentInfoUrl = studentFirebase+"student.json"
requests.put(url=studentInfoUrl, data=json.dumps(student)) # idempotent

moduleInfoUrl = moduleFirebase+"module.json"
requests.put(url=moduleInfoUrl, data=json.dumps(module))



<Response [200]>